In [ ]:
from math import exp, log

import numpy as np
import scipy as sp
from scipy import stats
from scipy.special import expit

import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV
import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
%matplotlib inline 
# %matplotlib notebook

from bokeh.io import output_notebook
from bokeh.plotting import figure, show, output_file
from bokeh.charts import HeatMap, bins

%load_ext autoreload
%autoreload 2

#output_notebook()

In [ ]:
np.random.seed(42)

In [ ]:
p1 = figure(title="Beta of Disease", background_fill_color="white")

x = np.linspace(0, 1, 1000)
beta_dist = stats.beta(0.5, 1)
rvs = beta_dist.rvs(size=10000)
pdf = beta_dist.pdf(x)

hist, edges = np.histogram(rvs, density=True, bins=50)

p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color=None, line_color="#033649")
p1.line(x, pdf, line_color="#D95B43", line_width=2, alpha=0.7, legend="PDF")

p1.legend.location = "top_left"
p1.xaxis.axis_label = 'x'
p1.yaxis.axis_label = 'Pr(x)'

show(p1);

In [ ]:
p1 = figure(title="Severity of Disease", background_fill_color="white")

x = np.linspace(0, 1, 1000)
sev_dist = stats.beta(3, 1.5)
rvs = sev_dist.rvs(size=10000)
pdf = sev_dist.pdf(x)

hist, edges = np.histogram(rvs, density=True, bins=50)

p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color=None, line_color="#033649")
p1.line(x, pdf, line_color="#D95B43", line_width=2, alpha=0.7, legend="PDF")

p1.legend.location = "top_left"
p1.xaxis.axis_label = 'x'
p1.yaxis.axis_label = 'Pr(x)'

show(p1);

In [ ]:
p2 = figure(title="Age in years", background_fill_color="white")

x = np.linspace(0, 100, 1000)
ages_dist = stats.gamma(8, scale=4)
rvs = ages_dist.rvs(size=10000)
pdf = ages_dist.pdf(x)

hist, edges = np.histogram(rvs, density=True, bins=50)

p2.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color=None, line_color="#033649")
p2.line(x, pdf, line_color="#D95B43", line_width=2, alpha=0.7, legend="PDF")

p2.legend.location = "top_left"
p2.xaxis.axis_label = 'x'
p2.yaxis.axis_label = 'Pr(x)'

show(p2);

##### Expected recovery time (days) = $exp(2+0.5*I_{male} + 0.03*age + 0.2*severity - 1*I_{medication})$

# Double-blind A/B test

In [ ]:
N = 100000

In [ ]:
sexes = np.random.randint(0,2, size=N)  # sex == 1 if male otherwise female
ages = ages_dist.rvs(size=N)
severties = sev_dist.rvs(size=N)
meds = np.random.randint(0,2, size=N)
const = np.ones(N)

In [ ]:
df = pd.DataFrame(dict(sex=sexes, age=ages, severity=severties, medication=meds, const=const))
features = ['sex', 'age', 'severity', 'medication', 'const']
df = df[features]
df.head()

In [ ]:
df.describe()

In [ ]:
data = df.corr()
sns.heatmap(data)

In [ ]:
data.reset_index()

In [ ]:
def exp_recovery_time(sex, age, severity, medication):
    return exp(2+0.5*sex+0.03*age+2*severity-1*medication)

In [ ]:
def rvs_recovery_time(sex, age, severity, medication, *args):
    return stats.poisson.rvs(exp_recovery_time(sex, age, severity, medication))

In [ ]:
exp_recovery_time(1, 50, 0.5, 1)

In [ ]:
df['recovery'] = df.apply(lambda x: rvs_recovery_time(*x) , axis=1)
df.head()

In [ ]:
glm = sm.GLM(df['recovery'], df[features], family=sm.families.Poisson())
res = glm.fit()
print(res.summary())

In [ ]:
reg = RandomForestRegressor()
X = df[features].as_matrix()
y = df['recovery'].values
reg.fit(X, y)

In [ ]:
X_neg = np.copy(X)
X_neg[:, df.columns.get_loc('medication')] = 0
X_pos = np.copy(X)
X_pos[:, df.columns.get_loc('medication')] = 1

In [ ]:
effect = np.mean(reg.predict(X_pos) / reg.predict(X_neg))

In [ ]:
np.log(effect)

# Now with a bias, likelihood of getting a treatment rises with the severity

In [ ]:
def get_medication(sex, age, severity, medication, *args):
    return int(1/3*sex + 2/3*severity + 0.15*np.random.randn() > 0.8)

In [ ]:
df2 = df.copy().drop('recovery', axis=1)
df2['medication'] = df2.apply(lambda x: get_medication(*x), axis=1)
df2['recovery'] = df2.apply(lambda x: rvs_recovery_time(*x), axis=1)
df2.describe(percentiles=np.arange(0.1,1,0.1))

In [ ]:
df2['medication'].sum()/df2['medication'].count()

In [ ]:
df2.corr()

In [ ]:
data = df2.corr()
sns.heatmap(data)

In [ ]:
glm = sm.GLM(df2['recovery'], df2[features], family=sm.families.Poisson())
res = glm.fit()
print(res.summary())

In [ ]:
reg = RandomForestRegressor()
X2 = df2[features].as_matrix()
y2 = df2['recovery'].values
reg.fit(X2, y2)

In [ ]:
X2_neg = np.copy(X2)
X2_neg[:, df.columns.get_loc('medication')] = 0
X2_pos = np.copy(X2)
X2_pos[:, df.columns.get_loc('medication')] = 1

In [ ]:
effect = np.mean(reg.predict(X2_pos) / reg.predict(X2_neg))

In [ ]:
np.log(effect)

### Oooppss, wrong coefficients!

# Let's use the propensity score

In [ ]:
features

In [ ]:
#cls = RandomForestClassifier()
cls = LogisticRegression(random_state=42)
cls = GaussianNB()
#cls = LinearSVC()

cls = CalibratedClassifierCV(cls)

X3 = df2[features].drop(['medication'], axis=1).as_matrix()
y3 = df2['medication'].values
cls.fit(X3, y3)

In [ ]:
propensity = pd.DataFrame(cls.predict_proba(X3))
propensity[propensity[0]==0.] = 1e-3
propensity[propensity[1]==0.] = 1e-3

In [ ]:
sp.stats.describe(propensity.lookup(np.arange(propensity.shape[0]), df2['medication'].values))

In [ ]:
propensity.hist(bins=100);

In [ ]:
propensity.describe()

In [ ]:
df2['iptw'] = 1. / propensity.lookup(np.arange(propensity.shape[0]), df2['medication'])

In [ ]:
propensity.lookup(np.arange(propensity.shape[0]), df2['medication'])[:20]

In [ ]:
df2.describe()

In [ ]:
glm = sm.GLM(df2['recovery'], df2[features], family=sm.families.Poisson(), freq_weights=df2['iptw'] )
res = glm.fit()
print(res.summary())

In [ ]:
reg = RandomForestRegressor(random_state=42)
X4 = df2[features].as_matrix()
y4 = df2['recovery'].values
reg.fit(X4, y4, sample_weight=df2['iptw'].values)

In [ ]:
X4_neg = np.copy(X4)
X4_neg[:, df.columns.get_loc('medication')] = 0
X4_pos = np.copy(X4)
X4_pos[:, df.columns.get_loc('medication')] = 1

In [ ]:
effect = np.mean(reg.predict(X4_pos) / reg.predict(X4_neg))

In [ ]:
np.log(effect)